In [5]:
import os
import re
import datetime
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By

from tika import parser
from bs4 import BeautifulSoup
import requests

In [6]:
last_page = 23
pages = np.arange(1,last_page +1)

path = "../data/"

if not os.path.exists(path):
    os.makedirs(path)

options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
prefs = {"profile.default_content_settings.popups": 0,
             "download.default_directory": 
                        path,
             "directory_upgrade": True}
options.add_experimental_option("prefs", prefs)
driver=webdriver.Chrome("./chromedriver", options=options)

for page in pages:
    url = 'http://www.bok.or.kr/portal/bbs/B0000245/list.do?menuNo=200761&pageIndex={}'.format(page)
    driver.get(url)
    
    li_tags = driver.find_elements(By.CSS_SELECTOR,'div.type2 li a.secretSet')
    n_li_tags = len(li_tags)
    for idx in range(n_li_tags):
        li_tag = li_tags[idx]
        li_tag.click()
        name = driver.find_element(By.CSS_SELECTOR,'#board > div > dl > dd > div > ul > li:nth-child(1) > a:nth-child(1)').text
        if name.split('.')[1] == 'hwp':
            try:
                driver.find_element(By.CSS_SELECTOR,'#board > div > dl > dd > div > ul > li:nth-child(2) > a:nth-child(1)').click()
            except:
                driver.find_element(By.CSS_SELECTOR,'#board > div > dl > dd > div > ul > li > a:nth-child(1)').click()
        else:
            driver.find_element(By.CSS_SELECTOR,'#board > div > dl > dd > div > ul > li:nth-child(1) > a:nth-child(1)').click()
        driver.back()
        li_tags = driver.find_elements(By.CSS_SELECTOR,'div.type2 li a.secretSet') 


/var/folders/85/178rxd5d5b31szqc8b3mznd40000gn/T/ipykernel_7520/3465109593.py:16: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome("./chromedriver", options=options)


In [7]:
path = '../data/'
file_list = os.listdir(path)

file_list_pdf = [file for file in file_list if file.endswith(".pdf")]
file_list_hwp = [file for file in file_list if file.endswith(".hwp")]

file_list = file_list_pdf + file_list_hwp
print(file_list[:10], len(file_list), sep='\n')

['금융통화위원회 의사록(2022년도 제14차).pdf', '금융통화위원회 의사록(2022년도 제18차).pdf', '제13차 금통위 의사록 (1).pdf', '제24차 금통위 의사록 (3).pdf', '제23차 금통위 의사록 (2).pdf', '제22차_금통위 의사록 (1).pdf', '제16차 금통위 의사록 (2).pdf', '제12차_금통위 의사록 (2).pdf', '제11차 금통위 의사록 (3).pdf', '제7차 금통위 의사록 (4).pdf']
230


In [8]:
dates = []
contents = []
titles = []

i = 1
for idx, file in enumerate(file_list):
    
    raw = parser.from_file('../data/{}'.format(file))
    date2 = re.search('\d{4}[가-힣\s]+\d+월? \d+일?', raw['content']).group()
    date2 = '.'.join(re.findall('[0-9]+',date2))
        
    title = file.split('.')[0]

    dates.append(date2)
    titles.append(title)
    contents.append(raw['content'])

    i += 1
    if i % 10 == 0:
        print(i,'/',len(file_list), '완료')
print('모든 작업 완료')

2022-11-24 09:46:50,889 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar to /var/folders/85/178rxd5d5b31szqc8b3mznd40000gn/T/tika-server.jar.
2022-11-24 09:47:01,228 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar.md5 to /var/folders/85/178rxd5d5b31szqc8b3mznd40000gn/T/tika-server.jar.md5.
2022-11-24 09:47:02,361 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


10 / 230 완료
20 / 230 완료
30 / 230 완료
40 / 230 완료
50 / 230 완료
60 / 230 완료
70 / 230 완료
80 / 230 완료
90 / 230 완료
100 / 230 완료
110 / 230 완료
120 / 230 완료
130 / 230 완료
140 / 230 완료
150 / 230 완료
160 / 230 완료
170 / 230 완료
180 / 230 완료
190 / 230 완료
200 / 230 완료
210 / 230 완료
220 / 230 완료
230 / 230 완료
모든 작업 완료


In [9]:
df = pd.DataFrame({'date':dates,'title':titles,'content':contents})

df.date = pd.to_datetime(df.date)
df = df.sort_values(by='date',ascending=True).reset_index(drop=True)

df

,date,title,content
0,2011-03-10,제7차 금통위 의사록 (6),\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
1,2011-03-24,제8차 금통위 의사록 (4),\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
2,2011-04-12,제9차 금통위 의사록 (5),\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
3,2011-05-13,제13차 금통위 의사록 (6),\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
4,2011-06-10,제15차 금통위 의사록 (5),\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
...,...,...,...
225,2022-08-25,금융통화위원회 의사록(2022년도 제16차),\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
226,2022-09-08,금융통화위원회 의사록(2022년도 제17차),\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
227,2022-09-22,금융통화위원회 의사록(2022년도 제18차),\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
228,2022-10-12,금융통화위원회 의사록(2022년도 제19차),\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...


In [10]:
df.to_csv('../data/bok_data.csv', encoding='utf-8-sig', header=True, index=False)